# Wrangling Data From Bay Area Bike Share Published Data - Weather

### Import Packages

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import datetime

import seaborn as sns
sns.set()

<p> Set some notebook variables, makes the notebook 95% width of the screen for easier viewing</p>

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

<p> Set some global font sizes for plots </p>

In [ ]:
font = {'size'   : 50}
matplotlib.rc('font', **font)

LABEL_FONT_SIZE = 15
TITLE_FONT_SIZE = 25

## Import Data

In [ ]:
print('Started Loading Weather Data...')
file_path_slug = '../source_data/bayareasbikeshare/*_weather_data.csv'
file_list = glob(file_path_slug)

weather_df = pd.DataFrame()

counter = 1
chunks = []

for file in file_list:
    for chunk in pd.read_csv(file, chunksize=10000, iterator=True):
        chunk.columns = ['Date', 'Max_Temperature_F', 'Mean_Temperature_F', 'Min_TemperatureF', 'Max_Dew_Point_F', 
                         'MeanDew_Point_F', 'Min_Dewpoint_F', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
                         'Max_Sea_Level_Pressure_In', 'Mean_Sea_Level_Pressure_In', 'Min_Sea_Level_Pressure_In', 
                         'Max_Visibility_Miles', 'Mean_Visibility_Miles', 'Min_Visibility_Miles', 
                         'Max_Wind_Speed_MPH', 'Mean_Wind_Speed_MPH', 'Max_Gust_Speed_MPH', 'Precipitation_In', 
                         'Cloud_Cover', 'Events', 'Wind_Dir_Degrees', 'zip']
        chunks.append(chunk)
    print('\tfinished file! (%d of %d)'% (counter, len(file_list)))
    counter += 1

weather_df = pd.concat(chunks)
print('Data Loaded Successfully!')

# Clean Data

In [ ]:
# clean column names, remove white spaces
weather_df.columns = [col.replace(' ', '_').lower() for col in weather_df.columns]

# convert dates to proper formats and datetime objects
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%m/%d/%Y')

# extract only columns we wish to keep
important_cols = ['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperaturef',
                  'max_wind_speed_mph', 'mean_wind_speed_mph', 'max_gust_speed_mph',
                  'precipitation_in', 'cloud_cover', 'events', 'zip']

for col in weather_df.columns:
    if col not in important_cols:
        weather_df.drop(col, axis=1, inplace=True)

# rename columns
weather_df.columns = ['date', 'max_temp', 'mean_temp', 'min_temp', 
                      'max_wind', 'mean_wind', 'max_gust', 'precipitation', 
                      'cloud_cover', 'events', 'zip_code']


# Sanitize 'prcipitation' column values
weather_df.precipitation = pd.to_numeric(weather_df.precipitation, errors='coerce')
weather_df.precipitation.fillna(0.0, inplace=True)

# sort columns by 'date' and 'zip_code' and set index
weather_df.sort_values(['date', 'zip_code'], inplace=True)
weather_df.reset_index(drop=True, inplace=True)

# Write Data to File

In [ ]:
weather_df.to_csv('../clean_data/bayareabikeshare/weather_data_cleaned.csv', encoding='utf-8')

***
# Preview EDA of Weather Data

In [ ]:
weather_df.set_index('date', inplace=True)

def plot_temperature_range(df_weather, df_median, df_mean, title, x_label='Date', y_label='Temperature (°F)'):
    
    weather_range = df_weather['max_temp'] - df_weather['min_temp']
    weather_r = weather_range.to_frame()
    weather_r.columns=['range']
    
    median_range = df_median['max_temp'] - df_median['min_temp']
    median_r = median_range.to_frame()
    median_r.columns=['range']
    
    mean_range = df_mean['max_temp'] - df_mean['min_temp']
    mean_r = mean_range.to_frame()
    mean_r.columns=['range']
    
    
    ax = mean_r['range'].plot(figsize=(12,6), label='mean_range', color='m')
    median_r['range'].plot(ax=ax, label='median_range', color='k', linestyle=':')
    weather_r['range'].plot(ax=ax, label='range', linestyle='None', marker='.', color='m', alpha=0.1)

    plt.legend(['mean', 'median'],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    ax.set_ylabel('Temperature Difference (°F)')
    ax.set_ylim(0, 40)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')
    ax.set_xlabel(x_label, size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel(y_label, size=LABEL_FONT_SIZE, weight='bold')
    plt.tight_layout()
    plt.show()
    
    
    ax = df_mean['max_temp'].plot(figsize=(12,6), label='max_rolling', color='r')
    df_mean['mean_temp'].plot(ax=ax, label='mean_rolling', color='k')
    df_mean['min_temp'].plot(ax=ax, label='min_rolling', color='b')

    df_median['max_temp'].plot(ax=ax, label='max_temp', color='r', linestyle=':', alpha=0.75)
    df_median['mean_temp'].plot(ax=ax, label='mean_temp', color='k', linestyle=':', alpha=0.75)
    df_median['min_temp'].plot(ax=ax, label='min_temp', color='b', linestyle=':', alpha=0.75)

    plt.legend(['max', 'mean', 'min'],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    ax.set_ylabel('Temperature (°F)')
    ax.set_ylim(20, 100)
    plt.title('Monthly Mean Temperature', size=TITLE_FONT_SIZE, weight='bold')
    ax.set_xlabel(x_label, size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel(y_label, size=LABEL_FONT_SIZE, weight='bold')
    plt.tight_layout()
    plt.show() 

### San Francisco Weather

In [ ]:
sanFrancisco_weather = weather_df[weather_df.zip_code == 94107]

# resample Mean
sf_weather_monthly_mean = sanFrancisco_weather.resample('M').mean()
sf_weather_monthly_rolling_mean = sanFrancisco_weather.resample('M').mean().fillna(0).rolling(window=3, min_periods=1).mean()

# resample Min
sf_weather_monthly_min = sanFrancisco_weather.resample('M').min()
sf_weather_monthly_rolling_min = sanFrancisco_weather.resample('M').min().fillna(0).rolling(window=3, min_periods=1).min()

# resample Max
sf_weather_monthly_max = sanFrancisco_weather.resample('M').max()
sf_weather_monthly_rolling_max = sanFrancisco_weather.resample('M').max().fillna(0).rolling(window=3, min_periods=1).max()

# resample Median
sf_weather_monthly_median = sanFrancisco_weather.resample('M').median()
sf_weather_monthly_rolling_median = sanFrancisco_weather.resample('M').median().fillna(0).rolling(window=3, min_periods=1).median()

plot_temperature_range(sanFrancisco_weather, sf_weather_monthly_rolling_median, sf_weather_monthly_rolling_mean, 'San Francisco Weather')

# Quick Reference to All Weather Data

In [ ]:
weather_df.head(10)